# MNIST Dataset 최적화
    - val_accuracy: 0.99
    - epochs 수
    - 모델 크기
    - dropout
    - batchnormalization
    
월요일 9시 전까지 제출: kgmyh@naver.com
노트북파일명 : 이름_mnist.ipynb
제목 : 이름 딥러닝 과제

In [30]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import random

from tensorflow.keras import layers

import matplotlib as mpl
import matplotlib.pyplot as plt


In [31]:
(train_image, train_label), (test_image, test_label) = keras.datasets.mnist.load_data()
train_image.shape, test_image.shape

((60000, 28, 28), (10000, 28, 28))

In [32]:
# 하이퍼파라미터, 변수 정의

LEARNING_RATE = 0.001   # 학습률 지정
N_EPOCHS = 40   # epoch수  데이터를 몇번쓸지(몇번 반복할지를 정하는 수)
N_BATCHS = 100    # 배치 사이즈

# 변수
N_TRAIN = train_X.shape[0]   # train 데이터셋의 데이터개수
N_TEST = test_X.shape[0]     # test 데이터셋의 데이터개수

In [33]:
# X, y 전처리, 

X_train = (train_image/255.0).astype(np.float32)
X_test = (test_image/255.0).astype(np.float32)

y_train = keras.utils.to_categorical(train_label)
y_test = keras.utils.to_categorical(test_label)



In [34]:
# Dataset 생성

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(N_TRAIN).batch(N_BATCHS)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(N_BATCHS)

In [38]:
def create_bn_model():
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28,28)))
    
    model.add(layers.Dense(units=1024)) #WX + b
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())#activation 함수
    
    
    model.add(layers.Dense(units=1024))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Dense(units=512))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Dense(units=512))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Dense(units=256))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Dense(units=256))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Dense(units=10)) #출력
    model.add(layers.BatchNormalization())
    model.add(layers.Softmax())
    return model

In [41]:
model = create_bn_model()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 1024)              803840    
_________________________________________________________________
batch_normalization_31 (Batc (None, 1024)              4096      
_________________________________________________________________
re_lu_26 (ReLU)              (None, 1024)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_32 (Batc (None, 1024)              4096      
_________________________________________________________________
re_lu_27 (ReLU)              (None, 1024)             

In [42]:
# optimizer에 고정된 learning rate 대신에 Learning Rate Scheduler를 생성해서 등록
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, # 초기(시작) 학습률
                                                           decay_steps= 600*5, # 몇 step마다 learing rate를 변경할 것인지 지정. (1에폭당 600스텝-600*5: 5에폭 당 LR를 변경)
                                                           decay_rate=0.5, # 변화율. 현재 Learning Rate * decay_rate = 새 Learing rate
                                                           staircase=True # 계단형태로 떨어트릴지(True), step마다 조금씩 떨어트릴지(False) 설정
                                                          )
model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_scheduler), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [43]:
hist = model.fit(train_dataset, epochs=N_EPOCHS, validation_data=test_dataset)

Epoch 1/40
600/600 [==============================] - 25s 39ms/step - loss: 0.3434 - accuracy: 0.9428 - val_loss: 0.1557 - val_accuracy: 0.9679
Epoch 2/40
600/600 [==============================] - 27s 44ms/step - loss: 0.1669 - accuracy: 0.9684 - val_loss: 0.1213 - val_accuracy: 0.9738
Epoch 3/40
600/600 [==============================] - 30s 50ms/step - loss: 0.1158 - accuracy: 0.9751 - val_loss: 0.0982 - val_accuracy: 0.9758
Epoch 4/40
600/600 [==============================] - 39s 65ms/step - loss: 0.0875 - accuracy: 0.9807 - val_loss: 0.1007 - val_accuracy: 0.9731
Epoch 5/40
600/600 [==============================] - 28s 46ms/step - loss: 0.0692 - accuracy: 0.9840 - val_loss: 0.0855 - val_accuracy: 0.9773
Epoch 6/40
600/600 [==============================] - 28s 46ms/step - loss: 0.0398 - accuracy: 0.9915 - val_loss: 0.0607 - val_accuracy: 0.9837
Epoch 7/40
600/600 [==============================] - 34s 56ms/step - loss: 0.0311 - accuracy: 0.9938 - val_loss: 0.0680 - val_accuracy: